## 0. Libarary 불러오기 및 경로설정

In [1]:
import os
import pandas as pd
import numpy as np
import datetime

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision

from PIL import Image
from copy import deepcopy
from tqdm.notebook import tqdm

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Set random seed
SEED = 2021
# random.seed(SEED)
np.random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)  # type: ignore
torch.backends.cudnn.deterministic = True  # type: ignore
torch.backends.cudnn.benchmark = True  # type: ignore

In [4]:
# 테스트 데이터셋 폴더 경로를 지정해주세요.
train_dir = '/opt/ml/input/data/train'

In [5]:
## HYPER PARAMETER 정의
EPOCHS = 10
BATCH_SIZE = 100
LEARNING_RATE = 0.001
CLASS_NUM = 18
IMAGE_SIZE = 224
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'{device} is using!')

cuda:0 is using!


## 1. ImageNet Pretrained Model을 torchvision에서 불러오기

In [6]:
resnet = torchvision.models.resnet18(pretrained=True)
print('네트워크 필요 입력 채널 개수', resnet.conv1.weight.shape[1])
print('네트워크 출력 채널 개수 (예측 class type 개수)', resnet.fc.weight.shape[0])
print(resnet)

네트워크 필요 입력 채널 개수 3
네트워크 출력 채널 개수 (예측 class type 개수) 1000
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, trac

In [7]:
# target model의 출력 크기를 변경하여 줍니다.
resnet.fc = torch.nn.Linear(in_features=512, out_features=CLASS_NUM, bias=True)

# 새롭게 넣은 네트워크 가중치를 xavier uniform으로 초기화
torch.nn.init.xavier_uniform_(resnet.fc.weight)
stdv = 1.0/np.sqrt(512)
resnet.fc.bias.data.uniform_(-stdv, stdv)

print('네트워크 출력 채널 개수 (예측 class type 개수)', resnet.fc.weight.shape[0])

네트워크 출력 채널 개수 (예측 class type 개수) 18


In [8]:
for param in resnet.parameters():
    param.requiers_grad = False

In [9]:
for param in resnet.fc.parameters():
    param.requires_grad = True

## 2. Train Dataset 정의

In [10]:
class TrainDataset(Dataset):
    def __init__(self, transform):
        train_info = pd.read_csv(os.path.join(train_dir, 'train_with_label.csv'))
        self.img_paths = train_info['image_path']
        self.transform = transform
        self.y = train_info['target']

    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self, idx):
        image = Image.open(self.img_paths[idx])
        label = self.y[idx]

        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(label)

In [11]:
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.CenterCrop(IMAGE_SIZE),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),
                         (0.5, 0.5, 0.5))
])

In [12]:
train_dataset = TrainDataset(transform)
len(train_dataset)

18900

## 3. Train DataLoader 정의

In [13]:
train_loader = torch.utils.data.DataLoader(train_dataset, 
                                           batch_size=BATCH_SIZE,
                                           shuffle=True,
                                           num_workers=8)

## 4. Criterion & Optimizer 정의

In [14]:
model = resnet.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

## 5. Train

In [15]:
for epoch in range(1, EPOCHS + 1):
    epoch_loss = 0
    epoch_acc = 0
    for idx, (X_batch, y_batch) in enumerate(tqdm(train_loader)):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device).type(torch.cuda.LongTensor)
        
        optimizer.zero_grad()
        
        y_pred = model.forward(X_batch)
        _, preds = torch.max(y_pred, 1)
        loss = criterion(y_pred, y_batch)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += torch.sum(preds == y_batch.data) / len(y_batch)

    epoch_loss = epoch_loss / len(train_loader)
    epoch_acc = epoch_acc / len(train_loader)
    
    print(f'현재 epoch-{epoch}의 데이터 셋에서 평균 Loss : {epoch_loss:.3f}, '
          f'평균 Accuracy : {epoch_acc:.3f}')

print('학습 종료!')


현재 epoch-1의 데이터 셋에서 평균 Loss : 0.469, 평균 Accuracy : 0.851



현재 epoch-2의 데이터 셋에서 평균 Loss : 0.217, 평균 Accuracy : 0.925



현재 epoch-3의 데이터 셋에서 평균 Loss : 0.150, 평균 Accuracy : 0.947



현재 epoch-4의 데이터 셋에서 평균 Loss : 0.102, 평균 Accuracy : 0.964



현재 epoch-5의 데이터 셋에서 평균 Loss : 0.089, 평균 Accuracy : 0.970



현재 epoch-6의 데이터 셋에서 평균 Loss : 0.058, 평균 Accuracy : 0.979



현재 epoch-7의 데이터 셋에서 평균 Loss : 0.056, 평균 Accuracy : 0.981



현재 epoch-8의 데이터 셋에서 평균 Loss : 0.045, 평균 Accuracy : 0.985



현재 epoch-9의 데이터 셋에서 평균 Loss : 0.054, 평균 Accuracy : 0.983



현재 epoch-10의 데이터 셋에서 평균 Loss : 0.044, 평균 Accuracy : 0.985
학습 종료!


## 6. Inference

In [16]:
# 테스트 데이터셋 폴더 경로를 지정해주세요.
test_dir = '/opt/ml/input/data/eval'

## 6.1 Test Dataset 정의

In [17]:
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),
                         (0.5, 0.5, 0.5))
])

In [18]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [19]:
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]

dataset = TestDataset(image_paths, transform)

loader = DataLoader(
    dataset,
    shuffle=False
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device = torch.device('cuda')
model = resnet
model.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in tqdm(loader):
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission_{}.csv'.format(
    datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'))), index=False)
print('test inference is done!')


test inference is done!
